In [11]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import gpxpy
from pyproj import Transformer

transformer = Transformer.from_crs("EPSG:4326", "EPSG:2056", always_xy=True)

In [5]:
filepath = "../data/7-gipfel-tour-flumserberg.gpx"

In [ ]:
gdf_waypoints = gpd.read_file(path_v2, layer="waypoints", driver="GPX")

gdf_waypoints = gdf_waypoints.set_crs(epsg=4326).to_crs(epsg=2056)

pts_3d = [Point(x, y, z) for x, y, z in zip(gdf_waypoints.geometry.x, gdf_waypoints.geometry.y, gdf_waypoints["ele"])]

gdf_waypoints = gdf_waypoints.set_geometry(pts_3d, crs=gdf_waypoints.crs)

gdf_waypoints = gdf_waypoints[["swisstopo_waypoint_id", "name", "geometry"]]

gdf_waypoints

,swisstopo_waypoint_id,name,geometry
0,1,Startpunkt,POINT Z (2737832.731 1214786.689 1974.2)
1,2,Ziel,POINT Z (2739850.838 1217402.568 1379.8)


In [ ]:


lines_lv95 = []

with open(path_v2, 'r', encoding='utf-8') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

    for track in gpx.tracks:
        for segment in track.segments:
            # für jeden Segment eine eigene Koordinatenliste
            coords = []
            for pt in segment.points:
                lon, lat, elev = pt.longitude, pt.latitude, pt.elevation or 0.0
                e, n = transformer.transform(lon, lat)
                coords.append((e, n, elev))
            # 3D-LineString erzeugen
            lines_lv95.append(LineString(coords))

# GeoDataFrame erstellen
gdf_lines = gpd.GeoDataFrame(
    { "geometry": lines_lv95 },
    crs="EPSG:2056"   # LV95
)

gdf_lines

,geometry
0,"LINESTRING Z (2737832.731 1214786.689 1974.2, ..."


In [49]:

## Erstellung Waypoints GeoDataFrame
gdf_waypoints = gpd.read_file(filepath, layer="waypoints", driver="GPX")
gdf_waypoints = gdf_waypoints.set_crs(epsg=4326).to_crs(epsg=2056)
pts_3d = [Point(x, y, z) for x, y, z in zip(gdf_waypoints.geometry.x, gdf_waypoints.geometry.y, gdf_waypoints["ele"])]
gdf_waypoints = gdf_waypoints.set_geometry(pts_3d, crs=gdf_waypoints.crs)[["swisstopo_waypoint_id", "name", "geometry"]]
gdf_waypoints.rename(columns={"swisstopo_waypoint_id": "id"}, inplace=True)

## Erstellung des Tracks GeoDataFrames
lines_lv95 = []
identifier_segment = 0
identifier_list = []

with open(filepath, 'r', encoding='utf-8') as gpx_file:
    gpx = gpxpy.parse(gpx_file)
    for track in gpx.tracks:
        for segment in track.segments:
            coords = []
            identifier_segment += 1
            for pt in segment.points:
                lon, lat, elev = pt.longitude, pt.latitude, pt.elevation or 0.0
                e, n = transformer.transform(lon, lat)
                coords.append((e, n, elev))
            # 3D-LineString erzeugen
            lines_lv95.append(LineString(coords))
            identifier_list.append(identifier_segment)

gdf_lines = gpd.GeoDataFrame(
    {"id": identifier_list, "geometry": lines_lv95},
    crs="EPSG:2056"   # LV95
)


## Erstellung des Ausgabe GeoDataFrames


In [50]:
gdf_waypoints


,id,name,geometry
0,1,Startpunkt,POINT Z (2737832.731 1214786.689 1974.2)
1,2,Ziel,POINT Z (2739850.838 1217402.568 1379.8)


In [51]:
gdf_lines

,id,geometry
0,1,"LINESTRING Z (2737832.731 1214786.689 1974.2, ..."


In [68]:
import geopandas as gpd
from shapely.geometry import Point

def attach_waypoints_to_lines(gdf_lines: gpd.GeoDataFrame,
                              gdf_waypoints: gpd.GeoDataFrame):

    ## Lines GeoDataFrame unbennene und angleichen an Waypoints GeoDataFrame
    gdf = (
        gdf_lines
        .rename(columns={"id": "segment_id"})
        .copy()
        .set_geometry("geometry")
        .rename_geometry("segment_geom")
        .to_crs(gdf_waypoints.crs)
    )

    ## Start und endpunkt extrahieren
    gdf["von_pkt_geom"] = gdf.segment_geom.apply(lambda ln: Point(*ln.coords[0]))
    gdf["bis_pkt_geom"] = gdf.segment_geom.apply(lambda ln: Point(*ln.coords[-1]))

    ## Erstellung der GeoDataFrames der Start und Endpunkte der Linien
    gdf_v = gdf[["segment_id", "von_pkt_geom"]].set_geometry("von_pkt_geom")
    gdf_b = gdf[["segment_id", "bis_pkt_geom"]].set_geometry("bis_pkt_geom")
    gdf_v.crs = gdf.crs
    gdf_b.crs = gdf.crs

    ## Joins erstellen
    sj_v = (
        gpd.sjoin_nearest(
            gdf_v,
            gdf_waypoints[["id", "name", "geometry"]],
            how="left"
        )
        .rename(columns={"id": "von_pkt_id", "name": "von_pkt_name"})
        .drop(columns=["index_right"])
    )

    sj_b = (
        gpd.sjoin_nearest(
            gdf_b,
            gdf_waypoints[["id", "name", "geometry"]],
            how="left"
        )
        .rename(columns={"id": "bis_pkt_id", "name": "bis_pkt_name"})
        .drop(columns=["index_right"])
    )

    ## Merge der Waypoints in die Linien GeoDataFrame
    df = (
        gdf
        .merge(sj_v[["segment_id", "von_pkt_name"]], on="segment_id", how="left")
        .merge(sj_b[["segment_id", "bis_pkt_name"]], on="segment_id", how="left")
    )

    ## Resultat-GeoDataFrame
    out = df[[
        "segment_id",
        "von_pkt_name", "von_pkt_geom",
        "bis_pkt_name", "bis_pkt_geom",
        "segment_geom"
    ]]

    out["segment_id"]     = out["segment_id"].astype("Int32")
    out["von_pkt_name"]   = out["von_pkt_name"].astype(str)
    out["bis_pkt_name"]   = out["bis_pkt_name"].astype(str)

    return gpd.GeoDataFrame(out, geometry="segment_geom", crs=gdf.crs)


In [69]:
result = attach_waypoints_to_lines(
    gdf_lines=gdf_lines,
    gdf_waypoints=gdf_waypoints
)

result.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   segment_id    1 non-null      Int32   
 1   von_pkt_name  1 non-null      object  
 2   von_pkt_geom  1 non-null      geometry
 3   bis_pkt_name  1 non-null      object  
 4   bis_pkt_geom  1 non-null      geometry
 5   segment_geom  1 non-null      geometry
dtypes: Int32(1), geometry(3), object(2)
memory usage: 177.0+ bytes
